In [1]:
import os
print(os.getcwd())

/home/mk009/DA6401/ae21b036_da6401_assignment-01/src/ann


In [2]:
import sys
sys.path.append(os.path.abspath(".."))

In [3]:
from neural_network import *

In [4]:
import argparse 

parser = argparse.ArgumentParser()
parser.add_argument('-d', choices=['mnist', 'fashion_mnist'], default= 'mnist', help= '--dataset: \'mnist\' or \'fashion_mnist\'')
parser.add_argument('-e', type= int, default= 1000, help= '--epochs: Number of training epochs')
parser.add_argument('-b', type= int, default= 64, help= '--batch_size: Mini-batch size')
parser.add_argument('-lr', type= float, default= 1e-3, help= '--learning_rate: Learning rate for optimizer')
parser.add_argument('-o', choices=['sgd', 'momentum', 'nag', 'rmsprop', 'adam', 'nadam'], default='sgd', help= '--optimizer: \'sgd\', \'momentum\', \'nag\', \'rmsprop\', \'adam\', \'nadam\'' )
parser.add_argument('-nhl', type=int, default=2, help= '--num_layers: Number of hidden layers')
parser.add_argument('-sz', type=int, nargs='+', help= '--hidden_size: Number of neurons in each hidden layer (list of values)')
parser.add_argument('-a', choices= ['sigmoid', 'tanh', 'relu'], default='sigmoid', help= '--activation: choice of sigmoid, tanh, relu')
parser.add_argument('-l', choices=['mean_squared_error', 'cross_entropy'], default='mean_squared_error', help= 'Loss function (\'cross_entropy\', \'mse\')')
parser.add_argument('-w_i', choices= ['random', 'xavier'], default='random', help= '--weight_init: choice of random or xavier')
parser.add_argument('-wd', type= float, default=0., help= '--weight_decay: Weight decay for L2 regularization')
parser.add_argument('-wb_project', default= '', help= '--wandb_project: W&B project name')
parser.add_argument('-save_path', default='', help= '--model_save_path: Path to save trained model')

_StoreAction(option_strings=['-save_path'], dest='save_path', nargs=None, const=None, default='', type=None, choices=None, required=False, help='--model_save_path: Path to save trained model', metavar=None)

In [5]:
args = parser.parse_args(args=[])

In [6]:
args.nhl = 2
args.sz = [512,512]
args.l = 'cross_entropy'
args.wd = 0.001
args.a = 'relu'

In [7]:
from tensorflow.keras import datasets

2026-02-23 13:29:20.103590: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-23 13:29:20.132276: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-23 13:29:21.127082: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [8]:
(x_train,y_train),(x_test,y_test) = datasets.mnist.load_data()

In [9]:
NN = NeuralNetwork(28*28, 10, Softmax, args)

In [10]:
c = np.copy(NN.Layers[0].W)

In [11]:
NN.train(x_train,y_train,1,512,True)


Epoch [1/1]


KeyboardInterrupt: 

In [23]:
NN.Layers[0].grad_W

array([[[-0.,  0., -0., ...,  0., -0.,  0.],
        [-0.,  0., -0., ...,  0., -0.,  0.],
        [-0.,  0., -0., ...,  0., -0.,  0.],
        ...,
        [-0.,  0., -0., ...,  0., -0.,  0.],
        [-0.,  0., -0., ...,  0., -0.,  0.],
        [-0.,  0., -0., ...,  0., -0.,  0.]],

       [[ 0., -0.,  0., ..., -0.,  0.,  0.],
        [ 0., -0.,  0., ..., -0.,  0.,  0.],
        [ 0., -0.,  0., ..., -0.,  0.,  0.],
        ...,
        [ 0., -0.,  0., ..., -0.,  0.,  0.],
        [ 0., -0.,  0., ..., -0.,  0.,  0.],
        [ 0., -0.,  0., ..., -0.,  0.,  0.]],

       [[ 0.,  0., -0., ..., -0.,  0., -0.],
        [ 0.,  0., -0., ..., -0.,  0., -0.],
        [ 0.,  0., -0., ..., -0.,  0., -0.],
        ...,
        [ 0.,  0., -0., ..., -0.,  0., -0.],
        [ 0.,  0., -0., ..., -0.,  0., -0.],
        [ 0.,  0., -0., ..., -0.,  0., -0.]],

       ...,

       [[ 0., -0., -0., ...,  0., -0., -0.],
        [ 0., -0., -0., ...,  0., -0., -0.],
        [ 0., -0., -0., ...,  0., -0., -0.